In [2]:
# # import cv2
# ##TODO : REMOVE VALIDATION SET- ISME convert into sets wale me dikkat aayegi
# ##TODO : DONT USE AUGMENTED DATA ON TRAINING SET
# ##TODO : LOOK INTO THE LAYERS WHERE ACT FUNC ARE RELU BUT IN MOBILENET V2 WE DONT USE RELU


# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.layers import Dense,BatchNormalization
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# 
# # Define paths to the finger vein dataset directory
# data_dir = "FVData"
# train_dir = "FVData/train"
# test_dir = "FVData/test"
# 
# # Image preprocessing function
# def preprocess_image(img):
#     return img / 255.0  # Normalize to the range [0, 1]
# 
# # Data generators
# batch_size = 32
# image_size = (224, 224)
# 
# #ImageDataGenerator is used for normalizing the pixel values to the range [0, 1],
# train_datagen = ImageDataGenerator(rescale=1./255)
# train_generator = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=image_size,
#     batch_size=batch_size,
#     class_mode='categorical',
#     shuffle=False  # Important: Set shuffle to False for feature extraction
# )
# 
# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(
#     test_dir,
#     target_size=image_size,
#     batch_size=batch_size,
#     class_mode='categorical',
#     shuffle=False  # Important: Set shuffle to False for feature extraction
# )
# 
# # Model definition for feature extraction with MobileNetV2 activations
# base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
# 
# for layer in base_model.layers:
#     layer.trainable = False
# 
# model = Sequential([
#     base_model,
#     tf.keras.layers.GlobalAveragePooling2D(),
#     Dense(256, activation="relu"),
#     layers.BatchNormalization(),
#     Dense(128, activation="relu"),
#     layers.BatchNormalization(),
#     Dense(64, activation="relu"),
#     layers.BatchNormalization()
# ])
# 
# # Extract features
# train_features = model.predict(train_generator)
# test_features = model.predict(test_generator)
# 
# # Save the features
# np.save("train_features.npy", train_features)
# np.save("test_features.npy", test_features)


import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense,BatchNormalization
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to the finger vein dataset directory
data_dir = "FVData"
train_dir = "Data/FVData/train"
test_dir = "Data/FVData/test"

# Image preprocessing function
def preprocess_image(img):
    return img / 255.0  # Normalize to the range [0, 1]

# Data generators
batch_size = 32
image_size = (224, 224)

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important: Set shuffle to False for feature extraction
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important: Set shuffle to False for feature extraction
)

# Model definition for feature extraction with MobileNetV2 activations
base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(256, activation="relu"),
    layers.BatchNormalization(),
    Dense(128, activation="relu"),
    layers.BatchNormalization(),
    Dense(64, activation="relu"),
    layers.BatchNormalization()
])

# Extract features
train_features = model.predict(train_generator)
test_features = model.predict(test_generator)

# Save the features and labels
np.save("train_features.npy", train_features)
np.save("train_labels.npy", train_generator.classes)
np.save("test_features.npy", test_features)
np.save("test_labels.npy", test_generator.classes)


Found 98170 images belonging to 96 classes.
Found 24518 images belonging to 96 classes.
767/767 [==============================] - 51s 67ms/step


In [30]:
# Assuming model is your MobileNetV2-based model
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 96)                6240      
                                                                 
Total params: 6,240
Trainable params: 6,240
Non-trainable params: 0
_________________________________________________________________


In [32]:
# # unknown_finger_path = "collected_final/2/2_1_0_0_1.bmp"
# # from tensorflow.keras.preprocessing import image
# # 
# # # Example usage for predicting an unknown finger vein image
# # # Load and preprocess the image
# # unknown_finger_img = image.load_img(unknown_finger_path, target_size=(224, 224))
# # unknown_finger_img_array = image.img_to_array(unknown_finger_img)
# # unknown_finger_img_array = preprocess_image(unknown_finger_img_array)
# # unknown_finger_img_array = np.expand_dims(unknown_finger_img_array, axis=0)
# # 
# # # Predict the class probabilities
# # predicted_class_probabilities = model.predict(unknown_finger_img_array)
# # 
# # # Get the predicted class index
# # predicted_class_index = np.argmax(predicted_class_probabilities, axis=1)[0]
# # 
# # # Map the class index to the actual class label using the data generator's class_indices
# # class_label = list(train_generator.class_indices.keys())[predicted_class_index]
# # 
# # print(f"Predicted class: {class_label}")
# 
# import numpy as np
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.losses import SparseCategoricalCrossentropy
# from tensorflow.keras.metrics import SparseCategoricalAccuracy
# from tensorflow.keras import regularizers
# 
# # Load the learned features
# train_features = np.load("train_features.npy")
# test_features = np.load("test_features.npy")
# 
# # Load the corresponding labels
# # Replace these paths with the actual paths to your label files
# train_labels = np.load("train_labels.npy")
# test_labels = np.load("test_labels.npy")
# 
# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)
# 
# # Define the classification model
# classifier_model = Sequential([
#     Dense(256, input_shape=(train_features.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
#     BatchNormalization(),
#     Activation("relu"),
#     Dropout(0.5),
# 
#     Dense(128, kernel_regularizer=regularizers.l2(0.01)),
#     BatchNormalization(),
#     Activation("relu"),
#     Dropout(0.5),
# 
#     Dense(64, kernel_regularizer=regularizers.l2(0.01)),
#     BatchNormalization(),
#     Activation("relu"),
#     Dropout(0.5),
# 
#     Dense(len(np.unique(train_labels)), activation="softmax")
# ])
# 
# # Compile the model
# classifier_model.compile(
#     optimizer=Adam(learning_rate=0.0001),
#     loss=SparseCategoricalCrossentropy(),
#     metrics=[SparseCategoricalAccuracy()]
# )
# 
# # Train the classification model
# classifier_model.fit(
#     X_train, y_train,
#     epochs=10,
#     batch_size=32,
#     validation_data=(X_val, y_val)
# )
# 
# # Evaluate the model on the test set
# test_results = classifier_model.evaluate(test_features, test_labels)
# print("Test Loss:", test_results[0])
# print("Test Accuracy:", test_results[1])
# 
# # Save the trained classifier model if needed
# # classifier_model.save("finger_vein_classifier_model.h5")
# 
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np

# Load the extracted features and labels
train_features = np.load("train_features.npy")
train_labels = np.load("train_labels.npy")
test_features = np.load("test_features.npy")
test_labels = np.load("test_labels.npy")

# Define the classifier model
def build_classifier_model(input_dim, num_classes):
    inputs = Input(shape=(input_dim,))
    x = Dense(256)(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes)(x)
    x = BatchNormalization()(x)
    predictions = Activation("softmax")(x)

    model = Model(inputs=inputs, outputs=predictions)
    return model

# Define the Center Loss layer
class CenterLossLayer(tf.keras.layers.Layer):
    def __init__(self, alpha, num_classes):
        super(CenterLossLayer, self).__init__()
        self.alpha = alpha
        self.num_classes = num_classes

    def build(self, input_shape):
        self.centers = self.add_weight(
            name='centers',
            shape=(self.num_classes, input_shape[1][-1]),
            initializer='uniform',
            trainable=True
        )

    def call(self, inputs, **kwargs):
        labels, features = inputs
        labels = tf.cast(labels, dtype=tf.int32)  # Ensure labels are of integer type
        labels = tf.reshape(labels, [-1])
        centers_batch = tf.gather(self.centers, labels)
        center_loss = tf.reduce_mean(tf.square(features - centers_batch))
        return center_loss


# Define the model with Center Loss and Cross-Entropy Loss
num_classes = len(np.unique(train_labels))
input_dim = train_features.shape[1]

classifier_model = build_classifier_model(input_dim, num_classes)

# Center Loss layer
alpha = 0.5  # Adjust as needed
center_loss_layer = CenterLossLayer(alpha, num_classes)([classifier_model.output, classifier_model.layers[-3].output])

# Final model with both losses
combined_model = Model(inputs=classifier_model.input, outputs=[classifier_model.output, center_loss_layer])

# Compile the model
combined_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'dense_46': 'categorical_crossentropy',
        'center_loss_layer_7': lambda y_true, y_pred: y_pred
    },
    loss_weights={
        'dense_3': 1.0,
        'center_loss_layer_7': alpha
    },
    metrics={'dense_3': 'accuracy'}
)




# Training the model
num_epochs = 10  # Adjust as needed
batch_size = 32  # Adjust as needed

history = combined_model.fit(train_features, [tf.keras.utils.to_categorical(train_labels - 1, num_classes), train_features],
                             epochs=num_epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
test_preds, _ = combined_model.predict(test_features)
test_preds_classes = np.argmax(test_preds, axis=1) + 1  # Adding 1 to shift labels to start from 1

# Calculate FAR and FRR
conf_matrix = confusion_matrix(test_labels, test_preds_classes)
far = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
frr = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
far = far / far.sum()
frr = frr / frr.sum()

# Calculate accuracy
accuracy = accuracy_score(test_labels, test_preds_classes)

print("FAR:", far)
print("FRR:", frr)
print("Accuracy:", accuracy)


Epoch 1/10


ValueError: in user code:

    File "C:\Users\ssnfs\anaconda3\envs\FVProject\Lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\ssnfs\anaconda3\envs\FVProject\Lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ssnfs\anaconda3\envs\FVProject\Lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\ssnfs\anaconda3\envs\FVProject\Lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\ssnfs\anaconda3\envs\FVProject\Lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "C:\Users\ssnfs\anaconda3\envs\FVProject\Lib\site-packages\keras\engine\compile_utils.py", 

In [14]:
import numpy as np

# Load labels from the .npy file
labels = np.load("train_labels.npy")

# Print the labels
print(labels)


[ 1  1  1 ... 96 96 96]


In [16]:
import numpy as np

# Load labels from the .npy file
feature = np.load("train_features.npy")

# Print the labels
print(feature)


[[0.         0.         0.4076832  ... 0.38587716 0.         0.7504778 ]
 [0.         0.         0.40484235 ... 0.47986877 0.         0.8896746 ]
 [0.         0.         0.35770246 ... 0.6519851  0.         0.86905146]
 ...
 [0.         0.         0.17601173 ... 0.5627908  0.         0.85993886]
 [0.         0.         0.18770634 ... 0.51039445 0.         1.052765  ]
 [0.         0.         0.10082483 ... 0.5466481  0.         1.0815744 ]]
